In [3]:
import os
import numpy as np

In [4]:
import pandas as pd
def create_pandaframe(path):
    d = np.load(path)
    return pd.DataFrame(d)
    

In [5]:
def generate_by_probability(data, n_new_rows):
    prob = np.mean(data, axis=0)
    prob= np.reshape(prob,[1,len(prob)])
    new_rows = np.random.rand(n_new_rows, data.shape[1]) < prob
    #t= np.where(new_rows==1)
    return new_rows.astype(int)

In [8]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def morph(sorce,target,nsor,ntar,item,root_folder):
    # Assuming sorce and target are pre-defined dataframes
    linhas_naotrucadas = np.ones(target.shape[0], dtype=bool)
    num_source_columns = sorce.shape[1] 
    num_target_columns = target.shape[1] 
    n_adicionar_c = abs((num_target_columns - num_source_columns) // 25)
    num_source_rows = sorce.shape[0] 
    num_target_rows = target.shape[0] 
    n_adicionar_r = abs((num_target_rows - num_source_rows) // 25)

    matrix = []

    for i in range(25):

        ad=generate_by_probability(sorce,n_adicionar_r)
        ad =pd.DataFrame(ad,columns=sorce.columns)
        sorce= pd.concat([sorce,ad],axis=0,ignore_index=True)
        num_source_columns = sorce.shape[1]
        num_source_rows = sorce.shape[0]

        X_train = target.iloc[:, :num_source_columns]
        y_train = target.iloc[:, num_source_columns:num_source_columns + n_adicionar_c]

        # Train the model
        model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10))
        model.fit(X_train, y_train)

        # Use the trained model to predict the next values of the columns in the source matrix
        source_predictions = model.predict(sorce)
        source_predictions_df = pd.DataFrame(source_predictions, index=sorce.index)

        # Concatenate source_predictions_df directly to sorce
        sorce = pd.concat([sorce, source_predictions_df], axis=1)

        print(sorce.shape)

        indices = np.where(linhas_naotrucadas)[0]
        indice_pos= np.where(indices<num_source_rows)[0]
        replace_indices = np.random.choice(indice_pos, n_adicionar_r-10 , replace=False)
        linhas_naotrucadas[replace_indices] = False
        print(len(indices))

        # Replace the chosen rows with corresponding rows from target
        sorce.iloc[replace_indices, :] = target.iloc[replace_indices, :sorce.shape[1]]
        create_folders_for_matrices(root_folder,nsor,ntar,sorce,item,i)
        del ad, X_train, y_train, model, source_predictions, source_predictions_df, indices, indice_pos, replace_indices
        import gc
        gc.collect()
    print("done")

In [9]:
import os
import shutil
def create_folders_for_matrices(root_folder,nsor,ntar,matrixs,item,n):
    new_datasets_folder = root_folder
    if not os.path.exists(new_datasets_folder):
        os.makedirs(new_datasets_folder)
    ui = './' + nsor + '/user_item.txt'
    bi = './' + nsor + '/bundle_item.txt'
    matrix_nam = nsor +"-" + ntar + str(n)
    matrix_folder = os.path.join(new_datasets_folder, matrix_nam)
    if not os.path.exists(matrix_folder):
        os.makedirs(matrix_folder)

     # Load the matrix
    matrix = matrixs.to_numpy()
    np.save(os.path.join(matrix_folder, "user_bundle_matrix.npy"),matrix)


    shutil.copy(ui, matrix_folder)
    shutil.copy(bi, matrix_folder)

    # Extract user IDs and bundle IDs where value is 1
    user_bundle_pairs = []
    for row_idx, row in enumerate(matrix):
        for col_idx, value in enumerate(row):
            if value == 1:
                user_bundle_pairs.append([row_idx, col_idx])

    # Save all_data.txt
    all_data_file = os.path.join(matrix_folder, "all_data.txt")
    np.savetxt(all_data_file, user_bundle_pairs, fmt='%d')

    # Calculate proportions for train, test, and tune
    num_rows = len(user_bundle_pairs)
    num_train = int(num_rows * 0.7)
    num_test = int(num_rows * 0.2)
    num_tune = num_rows - num_train - num_test

    # Shuffle the user/bundle pairs
    np.random.shuffle(user_bundle_pairs)

    size=np.zeros((1,3))
    size[0]=([len(matrix),len(matrix[0]),item])

    # Split the data
    train_data = user_bundle_pairs[:num_train]
    test_data = user_bundle_pairs[num_train:num_train + num_test]
    tune_data = user_bundle_pairs[num_train + num_test:]

        # Sort train_data by the first element ascending, and by the second element descending
    train_data_sorted = sorted(train_data, key=lambda x: (x[0], -x[1]))
    test_data = sorted(test_data, key=lambda x: (x[0], -x[1]))
    tune_data = sorted(tune_data, key=lambda x: (x[0], -x[1]))

    np.savetxt(os.path.join(matrix_folder, matrix_nam +"_data_size.txt"), size, fmt='%d', delimiter='\t')

        # Save train, test, and tune files with tab-separated values and sorted as required
    np.savetxt(os.path.join(matrix_folder, "user_bundle_train.txt"), train_data_sorted, fmt='%d', delimiter='\t')
    np.savetxt(os.path.join(matrix_folder, "user_bundle_test.txt"), test_data, fmt='%d', delimiter='\t')
    np.savetxt(os.path.join(matrix_folder, "user_bundle_tune.txt"), tune_data, fmt='%d', delimiter='\t')
    del matrix, user_bundle_pairs, train_data, test_data, tune_data
    import gc
    gc.collect()

In [10]:
root_folder = "C:/Users/35192/Desktop/codigo/codigos.py/projeto-ia-bundle/MultiCBR-main/datasets"
array={"Youshu": 32770,"NetEase":123628,"MealRec":10589,"iFashion":42563,"Steam" :2819 }
casos=[["MealRec","NetEase"],["MealRec","iFashion"],["Steam","iFashion"]]
array={"Youshu": 32770,"NetEase":123628,"MealRec":10589,"iFashion":42563,"Steam" :2819 }
casos=[["Steam","iFashion"]]
for i in casos:
    sorce = i[0]
    target = i[1]
    morph(create_pandaframe(os.path.join(root_folder, sorce, "user_bundle_matrix.npy")),
                  create_pandaframe(os.path.join(root_folder, target, "user_bundle_matrix.npy")),
                  sorce,target,array[target],root_folder)

KeyboardInterrupt: 